In [114]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

In [115]:
batch = db.batch()

df = pd.read_csv("/home/jupyter/spanner_shopify/apps.csv", sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv("/home/jupyter/spanner_shopify/pricing_plans.csv", sep=',', header=0, lineterminator='\n')
pricing_plans_rows = df.values.tolist()

df = pd.read_csv("/home/jupyter/spanner_shopify/key_benefits.csv", sep=',', header=0, lineterminator='\n')
key_benefits_rows = df.values.tolist()

df = pd.read_csv("/home/jupyter/spanner_shopify/reviews.csv", sep=',', header=0, lineterminator='\n')
reviews_rows = df.values.tolist()

counter = 0

for apps_row in apps_rows:
    counter += 1
    if counter >= 399:
        batch.commit()
        counter = 0
            
    apps_record = {}
    apps_record['id'] = apps_row[0]
    apps_record['url'] = apps_row[1]
    apps_record['title'] = apps_row[2]
    apps_record['developer'] = apps_row[3]
    apps_record['developer_link'] = apps_row[4]
    apps_record['icon'] = apps_row[5]
    apps_record['rating'] = apps_row[6]
    apps_record['reviews_count'] = apps_row[7]
    
    apps_ref = db.collection('apps').document(apps_row[0])
    
    batch.set(apps_ref, apps_record)
    
    for pricing_plans_row in pricing_plans_rows:
        if apps_row[0] == pricing_plans_row[1]:
            
            pricing_plans_record = {}
            pricing_plans_record['id'] = pricing_plans_row[0]
            pricing_plans_record['app_id'] = pricing_plans_row[1]
            pricing_plans_record['title'] = pricing_plans_row[2]
            pricing_plans_record['price'] = pricing_plans_row[3]
            #pricing_plans_record['feature'] = pricing_plans_row[4]
            
            pricing_plans_ref = apps_ref.collection('pricing_plans').document(pricing_plans_row[1]) # pricing_plans_row[0]
            
            batch.set(pricing_plans_ref, pricing_plans_record)
            counter += 1
            if counter >= 399:
                batch.commit()
                counter = 0
            
    for key_benefits_row in key_benefits_rows:
        if apps_row[0] == key_benefits_row[0]:
            
            key_benefits_record = {}
            key_benefits_record['title'] = key_benefits_row[1]
            key_benefits_record['description'] = key_benefits_row[2]
            #identifer = [key_benefits_row[0], key_benefits_row[1]]
            
            key_benefits_ref = apps_ref.collection('key_benefits').document(key_benefits_row[0])
            
            batch.set(key_benefits_ref, key_benefits_record)
            
            counter += 1
            if counter >= 399:
                batch.commit()
                counter = 0
            
    for reviews_row in reviews_rows:
        if apps_row[0] == reviews_row[0]:
            
            reviews_record = {}
            reviews_record['author'] = reviews_row[1]
            reviews_record['rating'] = reviews_row[2]
            reviews_record['posted_at'] = reviews_row[3]
            
            reviews_ref = apps_ref.collection('reviews').document(reviews_row[0])
            
            batch.set(reviews_ref, reviews_record)
            
            counter += 1
            if counter >= 399:
                batch.commit()
                counter = 0
            
    batch.commit()

In [116]:
batch = db.batch()

df = pd.read_csv("/home/jupyter/spanner_shopify/categories.csv", sep=',', header=0, lineterminator='\n')
categories_rows = df.values.tolist()

df = pd.read_csv("/home/jupyter/spanner_shopify/apps.csv", sep=',', header=0, lineterminator='\n')
#apps_rows = df.values.tolist()
ref = db.collection("apps")
data = ref.get()


for categories_row in categories_rows:
    
    categories_record = {}
    categories_record['app_id'] = categories_row[0]
    categories_record['category_id'] = categories_row[1]
    
    categories_ref = db.collection("categories").document(categories_row[0])
    
    batch.set(categories_ref, categories_record)
    
    
    for apps_row in data:
        if categories_row[0] == data[0]:
            '''
            apps_record = {}
            apps_record['url'] = apps_row[1]
            apps_record['title'] = apps_row[2]
            apps_record['developer'] = apps_row[3]
            apps_record['developer_link'] = apps_row[4]
            apps_record['icon'] = apps_row[5]
            apps_record['rating'] = apps_row[6]
            apps_record['reviews_count'] = apps_row[7]
            '''
            apps_ref = categories_ref.collection('apps').document(apps_row[0])
    
            batch.set(apps_ref, apps_rows.dict())
    
            batch.commit()
            

In [117]:
batch = db.batch()

collection = db.collection('apps')
documents = list(collection.get())
print("apps has {} documents".format(len(documents)))

apps has 3547 documents


In [118]:
batch = db.batch()

count = 0
for document in db.collection('apps').stream():
    plan = document.get('pricing_plans')
    count += len(plan)
print("pricing_plans has {} documents".format(len(count)))



KeyError: "'pricing_plans' is not contained in the data"

In [119]:
batch = db.batch()

count = 0
for document in db.collection('apps').stream():
    plan = document.get('key_benefits')
    count += len(plan)
print("key_benefits has {} documents".format(len(count)))

KeyError: "'key_benefits' is not contained in the data"

In [120]:
batch = db.batch()

count = 0
for document in db.collection('apps').stream():
    plan = document.get('reviews')
    count += len(plan)
print("reviews has {} documents".format(len(count)))

KeyError: "'reviews' is not contained in the data"

In [121]:
batch = db.batch()

count = 0
for document in db.collection('apps').stream():
    plan = document.get('categories')
    count += len(plan)
print("categories has {} documents".format(len(count)))

KeyError: "'categories' is not contained in the data"

In [122]:
collection = db.collection(u'apps')
query = collection.order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()
for app in results:
    #print(app.id, app.title, app.developer, app.rating, app.review_count)
    d = app.to_dict()
    print(app.id+", "+str(d['title'])+", "+str(d['developer'])+", "+str(d['rating'])+", "+str(d['reviews_count']))

d9f142ee-b141-4dc4-9353-173db61d2eb0, Privy ‑ Exit Pop Ups & Email, Privy, 4.7, 23078
78ea0810-c008-4a4e-a82f-de0c790e3286, Free Shipping Bar, Hextom, 4.9, 8737
b88488b0-9912-44d3-b736-224c36f09d95, Sales Pop ‑ Popup Notification, CartKit, 4.8, 6905
e528a60e-94f8-4e92-80e2-5bc6013b8283, BEST Currency Converter, Grizzly Apps, 4.8, 5986
be2640c4-01b5-4d52-9f68-cae8c0734d0d, Recart FB Messenger Marketing, Recart, 4.8, 5596
70bff9e0-4316-4cc6-84ce-92fcd1bc6925, EU Cookie Bar ‑ Cookie GDPR, Booster Apps, 4.7, 5259
171816e2-27d4-4552-a65e-ab44a312fe04, Sales Pop Master ‑ Countdown, Autoketing, 4.8, 4931
9025eff0-d714-4df1-930f-43f5582979ad, Ultimate Sales Boost, Hextom, 4.8, 4847
404183ac-6d52-4960-85d2-8350f37c2f0b, Ads by Varinode, Varinode, Inc., 4.9, 4245
0380c5ed-7b91-4d4b-9a4e-f6a1c50269dd, Quick Announcement Bar, Hextom, 4.9, 4237
